# Copied from 03b

In [1]:
import os
import re
import math
import pickle
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from def_colors import map_DMC, dmc_color, dft_color

from define_setup import *

# Read mol_info
with open( 'mol_info.pkl', 'rb' ) as f:
    mol_info = pickle.load( f )
#mol_info

df_mol_info = pd.read_csv( 'mol_info.csv', index_col=0 )
df_mol_info

# Read results monomer
with open( dir_analysis+'results_mol.pkl', 'rb' ) as f:
    results_mol = pickle.load( f )

# Read mol_info
with open( 'dim_info.pkl', 'rb' ) as f:
    dim_info = pickle.load( f )

dim_info

with open( dir_analysis+'results_dim.pkl', 'rb' ) as f:
    results_dim = pickle.load( f )

# results_dim[ dim_id ][ dmc_type ][ tau ] 
# results_dim[1]#['DMCdla5']['0.01']

In [2]:
df_refs = pd.read_csv( 'Refs.csv', index_col=0 )
df_refs

,name,Hobza_1,Hobza_2,Martin_newBronze,Martin_Silver,Martin_Gold,Martin_PCCP2022,14k-Gold
1,01 Water-Water,-4.92,-5.01,-4.915,-4.982,-4.979,-4.946,-4.991
2,02 Water-MeOH,-5.59,-5.70,-5.589,-5.666,-5.666,-5.627,-5.674
3,03 Water-MeNH2,-6.91,-7.04,-6.894,-6.986,-6.985,-6.941,-6.996
4,04 Water-Peptide,-8.10,-8.22,-8.089,-8.183,NaN,-8.128,-8.191
5,05 MeOH-MeOH,-5.76,-5.85,-5.773,-5.822,-5.824,-5.804,-5.827
...,...,...,...,...,...,...,...,...
62,62 Pentane-AcNH2,-3.53,-3.53,-3.458,-3.491,NaN,-3.437,-3.495
63,63 Benzene-AcOH,-3.80,-3.75,-3.715,-3.709,NaN,-3.671,-3.716
64,64 Peptide-Ethene,-3.00,-3.00,-2.945,-2.967,NaN,-2.928,-2.977
65,65 Pyridine-Ethyne,-3.99,-4.10,-4.065,-4.064,NaN,-4.066,-4.071


In [3]:
# Reading the evaluation in 03a_* 
# of DMC binding energies of all dimers, i=1,...,66, 
# into a Pamdas DataFrame, reporting Eb, error, dmc_type, dmc_Jas, name, ...
# Final DataFrame is obtained from the concatenation of individual dataframes,
# each for a different dimer and a different DMC setup


Edimer = pd.read_csv('Edimer.csv', index_col=0)

Edimer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366 entries, 0 to 365
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ene             366 non-null    float64
 1   err             366 non-null    float64
 2   units           366 non-null    object 
 3   TotTimeSim[au]  366 non-null    float64
 4   Pop             366 non-null    float64
 5   Pop-err         366 non-null    float64
 6   Var[au]         366 non-null    float64
 7   tau_eff[au]     366 non-null    float64
 8   corr-time[au]   366 non-null    float64
 9   machine         344 non-null    object 
 10  N_MPI           344 non-null    float64
 11  cost            344 non-null    float64
 12  target_weight   344 non-null    float64
 13  tau             366 non-null    float64
 14  name            366 non-null    object 
 15  i               366 non-null    int64  
 16  dmc_type        366 non-null    object 
 17  Jas             366 non-null    obj

In [4]:
Edimer[ Edimer.name == '01_Water-Water' ]

,ene,err,units,TotTimeSim[au],Pop,Pop-err,Var[au],tau_eff[au],corr-time[au],machine,N_MPI,cost,target_weight,tau,name,i,dmc_type,Jas,Nelv
0,-21596.426587,0.027148,kcal/mol,1050.0,63763.011538,1.264251,0.584035,0.085419,0.199963,linuxpc-intel-slurm-parallel.peta4,64.0,0.915718,64000.0,0.100,01_Water-Water,1,DMCtm5,Jopt,16
1,-21593.921876,0.023828,kcal/mol,1230.0,63869.469827,0.868089,0.566135,0.054910,0.160076,linuxpc-intel-slurm-parallel.peta4,64.0,0.929386,64000.0,0.060,01_Water-Water,1,DMCtm5,Jopt,16
2,-21593.206610,0.019761,kcal/mol,1215.0,63953.916472,0.655075,0.552120,0.028862,0.131564,linuxpc-intel-slurm-parallel.peta4,64.0,0.902262,64000.0,0.030,01_Water-Water,1,DMCtm5,Jopt,16
3,-21593.902714,0.022036,kcal/mol,1005.0,63984.356412,0.414554,0.540201,0.009906,0.125939,linuxpc-intel-slurm-parallel.peta4,64.0,0.883902,64000.0,0.010,01_Water-Water,1,DMCtm5,Jopt,16
4,-21594.285384,0.024074,kcal/mol,777.0,63982.548927,0.363725,0.535780,0.005972,0.135900,NaN,NaN,NaN,NaN,0.006,01_Water-Water,1,DMCtm5,Jopt,16
5,-21594.598270,0.038373,kcal/mol,388.5,64011.054103,0.368187,0.534968,0.002994,0.124343,NaN,NaN,NaN,NaN,0.003,01_Water-Water,1,DMCtm5,Jopt,16
6,-21596.239471,0.021220,kcal/mol,1230.0,63784.972950,0.853987,0.571911,0.054913,0.153619,linuxpc-intel-slurm-parallel.peta4,64.0,0.905245,64000.0,0.060,01_Water-Water,1,DMCtm5,Jopt,16
7,-21594.742661,0.021172,kcal/mol,1215.0,63898.234203,0.615966,0.572748,0.028863,0.134879,linuxpc-intel-slurm-parallel.peta4,64.0,0.892656,64000.0,0.030,01_Water-Water,1,DMCtm5,Jopt,16
8,-21594.575269,0.022685,kcal/mol,1010.0,63929.393700,0.567137,0.552755,0.019541,0.126479,linuxpc-intel-slurm-parallel.peta4,64.0,0.916368,64000.0,0.020,01_Water-Water,1,DMCtm5,Jopt,16
9,-21594.658280,0.020500,kcal/mol,1005.0,63963.003226,0.419906,0.548701,0.009906,0.117862,linuxpc-intel-slurm-parallel.peta4,64.0,0.891023,64000.0,0.010,01_Water-Water,1,DMCtm5,Jopt,16


In [5]:
df_mol_info

,ref,test,name,formula,atoms,Nel,Nelv
mol,,,,,,,
AcNH2,21_1,True,acetamide,H3C-CO-NH2,"{'H': 5, 'C': 2, 'N': 1, 'O': 1}",32,24
AcOH,20_1,True,acetic acid,H3C-COOH,"{'H': 4, 'C': 2, 'N': 0, 'O': 2}",32,24
Benzene,24_1,False,benzene,C6H6,"{'H': 6, 'C': 6, 'N': 0, 'O': 0}",42,30
Cyclopentane,37_1,True,cyclopentane,(CH2)5,"{'H': 10, 'C': 5, 'N': 0, 'O': 0}",40,30
Ethene,30_2,False,ethene,H2C=CH2,"{'H': 4, 'C': 2, 'N': 0, 'O': 0}",16,12
Ethyne,32_2,False,ethyne,HC---CH,"{'H': 2, 'C': 2, 'N': 0, 'O': 0}",14,10
MeNH2,56_2,False,methylamine,H3C-NH2,"{'H': 5, 'C': 1, 'N': 1, 'O': 0}",18,14
MeOH,55_2,False,methanol,H3C-OH,"{'H': 4, 'C': 1, 'N': 0, 'O': 1}",18,14
Neopentane,36_1,False,neopentane,C(CH3)4,"{'H': 12, 'C': 5, 'N': 0, 'O': 0}",42,32


In [6]:
df_mol_info.describe()

,Nel,Nelv
count,14.000000,14.000000
mean,31.857143,23.714286
std,14.303615,10.343549
min,10.000000,8.000000
25%,18.000000,14.000000
50%,36.000000,27.000000
75%,42.000000,30.000000
max,58.000000,42.000000


In [7]:
df_mol_info.sort_values(['Nelv','Nel','name'])

,ref,test,name,formula,atoms,Nel,Nelv
mol,,,,,,,
Water,01_2,False,water,H2O,"{'H': 2, 'C': 0, 'N': 0, 'O': 1}",10,8
Ethyne,32_2,False,ethyne,HC---CH,"{'H': 2, 'C': 2, 'N': 0, 'O': 0}",14,10
Ethene,30_2,False,ethene,H2C=CH2,"{'H': 4, 'C': 2, 'N': 0, 'O': 0}",16,12
MeOH,55_2,False,methanol,H3C-OH,"{'H': 4, 'C': 1, 'N': 0, 'O': 1}",18,14
MeNH2,56_2,False,methylamine,H3C-NH2,"{'H': 5, 'C': 1, 'N': 1, 'O': 0}",18,14
AcNH2,21_1,True,acetamide,H3C-CO-NH2,"{'H': 5, 'C': 2, 'N': 1, 'O': 1}",32,24
AcOH,20_1,True,acetic acid,H3C-COOH,"{'H': 4, 'C': 2, 'N': 0, 'O': 2}",32,24
Cyclopentane,37_1,True,cyclopentane,(CH2)5,"{'H': 10, 'C': 5, 'N': 0, 'O': 0}",40,30
Peptide,57_2,True,peptide,H3C-CO-NH-CH3,"{'H': 7, 'C': 3, 'N': 1, 'O': 1}",40,30


In [12]:
df_mol_info.max(1)

/tmp/ipykernel_9241/2012400720.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mol_info.max(1)


mol
AcNH2           32.0
AcOH            32.0
Benzene         42.0
Cyclopentane    40.0
Ethene          16.0
Ethyne          14.0
MeNH2           18.0
MeOH            18.0
Neopentane      42.0
Pentane         42.0
Peptide         40.0
Pyridine        42.0
Uracil          58.0
Water           10.0
dtype: float64

In [13]:
indexer = list(df_mol_info.max(1).argsort())
print(indexer)
df_mol_info.take(indexer)

[13, 5, 4, 6, 7, 0, 1, 3, 10, 2, 8, 9, 11, 12]


/tmp/ipykernel_9241/1468935217.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  indexer = list(df_mol_info.max(1).argsort())


,ref,test,name,formula,atoms,Nel,Nelv
mol,,,,,,,
Water,01_2,False,water,H2O,"{'H': 2, 'C': 0, 'N': 0, 'O': 1}",10,8
Ethyne,32_2,False,ethyne,HC---CH,"{'H': 2, 'C': 2, 'N': 0, 'O': 0}",14,10
Ethene,30_2,False,ethene,H2C=CH2,"{'H': 4, 'C': 2, 'N': 0, 'O': 0}",16,12
MeNH2,56_2,False,methylamine,H3C-NH2,"{'H': 5, 'C': 1, 'N': 1, 'O': 0}",18,14
MeOH,55_2,False,methanol,H3C-OH,"{'H': 4, 'C': 1, 'N': 0, 'O': 1}",18,14
AcNH2,21_1,True,acetamide,H3C-CO-NH2,"{'H': 5, 'C': 2, 'N': 1, 'O': 1}",32,24
AcOH,20_1,True,acetic acid,H3C-COOH,"{'H': 4, 'C': 2, 'N': 0, 'O': 2}",32,24
Cyclopentane,37_1,True,cyclopentane,(CH2)5,"{'H': 10, 'C': 5, 'N': 0, 'O': 0}",40,30
Peptide,57_2,True,peptide,H3C-CO-NH-CH3,"{'H': 7, 'C': 3, 'N': 1, 'O': 1}",40,30


In [14]:
df_mol_info.take(indexer).loc[:,'Nel':]

,Nel,Nelv
mol,,
Water,10,8
Ethyne,14,10
Ethene,16,12
MeNH2,18,14
MeOH,18,14
AcNH2,32,24
AcOH,32,24
Cyclopentane,40,30
Peptide,40,30


In [15]:
tested = np.where( df_mol_info['test']==True, 'tested', 'not tested')
tested

array(['tested', 'tested', 'not tested', 'tested', 'not tested',
       'not tested', 'not tested', 'not tested', 'not tested',
       'not tested', 'tested', 'not tested', 'tested', 'not tested'],
      dtype='<U10')

In [16]:
g = df_mol_info.groupby( [ 'Nelv', tested ] )
g.size().unstack().fillna(0)

,not tested,tested
Nelv,,
8,1.0,0.0
10,1.0,0.0
12,1.0,0.0
14,2.0,0.0
24,0.0,2.0
30,2.0,2.0
32,2.0,0.0
42,0.0,1.0


---

In [12]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                          "bar", "bar", "bar", "bar"],
                    "B": ["one", "one", "one", "two", "two",
                          "one", "one", "two", "two"],
                    "C": ["small", "large", "large", "small",
                          "small", "large", "small", "small",
                          "large"],
                    "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                    "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df


,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [15]:
table = pd.pivot_table(df, values='D', index=['A', 'B'],
                     columns=['C'], aggfunc=np.sum, fill_value=0)
table

C        large  small
A   B                
bar one      4      5
    two      7      6
foo one      4      1
    two      0      6

In [18]:
table.query('large>3 & small>2')

C        large  small
A   B                
bar one      4      5
    two      7      6

In [33]:
table.loc['bar','large']['one']

4

In [37]:
table.loc['bar','one']['large']

4

In [35]:
table.loc[:,'large']

A    B  
bar  one    4
     two    7
foo  one    4
     two    0
Name: large, dtype: int64

In [19]:
table.unstack()

C   large     small    
B     one two   one two
A                      
bar     4   7     5   6
foo     4   0     1   6

In [22]:
table.unstack().loc['bar','large']['one']

4

In [38]:
table.apply(lambda x: (x.large + x.small)/2, axis=1 )

A    B  
bar  one    4.5
     two    6.5
foo  one    2.5
     two    3.0
dtype: float64